In [1]:
NET = 'lstm3'
import os.path
import shutil
import sys
import tensorflow as tf
import datasets
import pickle
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from matplotlib.font_manager import FontProperties

if NET == 'dilation':
    import experiments.dilation as experiments
if NET == 'segnet':
    import experiments.segnet as experiments
elif NET == 'lstm':
    import experiments.lstm as experiments
elif NET == 'coloring':
    import experiments.coloring as experiments
elif NET == 'crossing':
    import experiments.crossing as experiments
elif NET == 'unet':
    import experiments.unet as experiments
elif NET == 'multi_lstm':
    import experiments.multi_lstm as experiments
elif NET == 'multi_lstm_init':
    import experiments.multi_lstm_init as experiments
elif NET == 'FF':
    import experiments.FF as experiments
elif NET == 'optimal_lstm':
    import experiments.optimal_lstm as experiments
elif NET == 'lstm3':
    import experiments.LSTM3 as experiments


        

        id: 0,
        name: DATA_CatNS0_D0,
        dataset_path: ,
        num_images_training: 1000.0,
        num_images_testing: 10000.0,
        type: ['NS0'],
        

        

        id: 1,
        name: DATA_CatNS2_D0,
        dataset_path: ,
        num_images_training: 1000.0,
        num_images_testing: 10000.0,
        type: ['NS2'],
        

        

        id: 2,
        name: DATA_CatNS4_D0,
        dataset_path: ,
        num_images_training: 1000.0,
        num_images_testing: 10000.0,
        type: ['NS4'],
        

        

        id: 3,
        name: DATA_CatNS6_D0,
        dataset_path: ,
        num_images_training: 1000.0,
        num_images_testing: 10000.0,
        type: ['NS6'],
        

        

        id: 4,
        name: DATA_CatNSd4_D0,
        dataset_path: ,
        num_images_training: 1000.0,
        num_images_testing: 10000.0,
        type: ['NSd4'],
        

        

        id: 5,
        name: DATA_CatS0_D0,
        datas

In [2]:
output_path = '/om/user/shobhita/data/symmetry/' + NET + '/'
network_id = 90 #specific ID we want to get activations for
run_opt = experiments.get_best_of_the_family(output_path, network_id)
opt_datasets = datasets.get_datasets(output_path)

215


In [3]:
SYMMETRIC_DATASETS = [opt_datasets[28]]
ASYMMETRIC_DATASETS = [opt_datasets[23]]

In [4]:
import gc
gc.collect()

0

In [4]:
datasets = SYMMETRIC_DATASETS
data_points = {ts: [] for ts in [49]}
for opt_data in datasets:
    with open(run_opt.log_dir_base + run_opt.name + '/results/activations_DATA' + opt_data.log_name + '.pkl', 'rb') as f:
        data_point = pickle.load(f)

In [24]:
len(data_point[0][0][0])

50

In [5]:
for datasets in [SYMMETRIC_DATASETS, ASYMMETRIC_DATASETS]:
    data_points = {ts: [] for ts in [9, 19, 29, 39, 49]}
    for opt_data in datasets:
        with open(run_opt.log_dir_base + run_opt.name + '/results/activations_DATA' + opt_data.log_name + '.pkl', 'rb') as f:
            data_point = pickle.load(f)
            for channel in range(64):
                for timestep in [9, 19, 29, 39, 49]:
                    for iter in range(0, 10, 2):
                            data_points[timestep].append(-data_point[iter][0][5][timestep][:, :, :, channel])
        print('--------------------')
        print(run_opt.name)
        print(opt_data.log_name)
        sys.stdout.flush()

    for timestep in [9, 19, 29, 39, 49]:
        print("\nCLUSTERING TIMESTEP {}".format(timestep))
        dp = data_points[timestep]
        outputs = np.concatenate(dp)
        print("OUTPUT SHAPE", outputs.shape)
        sys.stdout.flush()

        o = outputs.reshape((len(outputs), 400))

        df = pd.DataFrame(o)

        print("DF SHAPE: ", df.shape)
        sys.stdout.flush()

        n = 10
        kmeans = KMeans(n_clusters=n, random_state=0).fit(df)
        results = {}
        results["centroids"] = kmeans.cluster_centers_
        results["data"] = df
        results["samples"] = [outputs[15], outputs[45], outputs[53]]
        results["labels"] = kmeans.labels_
        results["inertia"] = kmeans.inertia_

        idx = "S" if datasets == SYMMETRIC_DATASETS else "NS"
        pickle.dump(results, open(output_path+"activation_clusters/activation_clusters_km_{}_timestep_{}_{}_d{}.pkl".format(n, timestep, idx, datasets[0].ID), "wb"))

--------------------
ID90_LSTM3_D32
ID28_DATA_CatS6_D2

CLUSTERING TIMESTEP 9
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 19
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 29
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 39
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 49
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)
--------------------
ID90_LSTM3_D32
ID23_DATA_CatNS6_D2

CLUSTERING TIMESTEP 9
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 19
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 29
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 39
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)

CLUSTERING TIMESTEP 49
OUTPUT SHAPE (10240, 20, 20)
DF SHAPE:  (10240, 400)
